In [1]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score,precision_score, recall_score, roc_auc_score
from sklearn.decomposition import PCA
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.mixture import GaussianMixture
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
import tensorflow.keras
import keras.metrics
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.callbacks import History 
from keras.utils import plot_model
from keras.optimizers import SGD


C:\Anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
path = "PTSD.xlsx"
df = pd.read_excel(path)
df = df[~df["PCL_Strict3"].isna()]


In [3]:
features =  [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD", "T1ETBE", "T1Acc1t",
                         "T1Acc1n", "T1bias", "T2Acc1t", "T2Acc1n", "T2bias", "state1", "state2", "trait1",
                         "trait2", "lot1", "lot2", "phq1", "phq2", "PCL1", "PCL2", "cd_risc1", "ptgi2",
                         "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                         "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                         "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                         "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                         "religion2", "emotional_support2", "instrumental_support2","self_distraction2",
                         "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                         "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                         "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                         "Sephar_scale", "Unknown"]

In [4]:
numerical_features = [ "T1ETBE", "T1Acc1t", "T1Acc1n", "T1bias", "T2Acc1t","T2Acc1n", "T2bias", "state1", "state2",
                        "trait1", "trait2", "lot1", "lot2", "phq1", "phq2", "cd_risc1", "PCL1", "PCL2"]
categorical_features = [ "age", "highschool_diploma", "Hebrew", "dyslexia", "ADHD",   "ptgi2",
                    "active_coping1", "planning1", "positive_reframing1", "acceptance1", "humor1",
                    "religion1", "emotional_support1", "instrumental_support1", "self_distraction1",
                    "denial1", "venting1", "substance_use1", "behavioral_disengagement1", "self_blame1",
                    "active_coping2", "planning2", "positive_reframing2", "acceptance2", "humor2",
                    "religion2", "emotional_support2", "instrumental_support2", "self_distraction2",
                    "denial2", "venting2", "substance_use2", "behavioral_disengagement2", "self_blame2",
                    "trauma_history8_1", "military_exposure_unit", "HML_5HTT", "HL_MAOA", "HML_NPY",
                    "COMT_Hap1_recode", "COMT_Hap2_recode", "COMT_Hap1_LvsMH", "HML_FKBP5", "Ashken_scale",
                    "Sephar_scale", "Unknown"]
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
df[numerical_features] = imp.fit_transform(df[numerical_features])

imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
df[categorical_features] = imp.fit_transform(df[categorical_features])

X = df[features]
X = X - X.mean()
Y = df["PCL_Strict3"]

In [5]:
X["interaction_1"] = X["T1Acc1t"] * X["T2Acc1n"] * X["military_exposure_unit"]
X["interaction_2"] = X["T1Acc1n"] * X["T2Acc1t"] * X["military_exposure_unit"]
X["interaction_3"] = X["highschool_diploma"] * X["military_exposure_unit"] * X['PCL1']
X["interaction_4"] = X["T1ETBE"] * X["military_exposure_unit"] * X['HML_5HTT']

# for i, feature in enumerate(features):
#     for interation in features[i::]:
#         X[f"interaction_{feature}_{interation}"] = X[feature]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.1, random_state=271828, stratify=Y)
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train, y_train, test_size = 0.1, random_state=271828, stratify=y_train)
#X_train_3, X_test_3, y_train_3, y_test_3 = train_test_split(X_train_2, y_train_2, test_size = 0.1, random_state=271828, stratify=y_train_2)

In [7]:
def create_data(X_train, y_train):
    X_train_3 = X_train[y_train==1]
    y_train_3 = y_train[y_train==1]
    X_train_4 = X_train[y_train==0][:20:]
    y_train_4 = y_train[y_train==0][:20:]
    X_train_5 = np.vstack((X_train_4, X_train_3))
    y_train_5 =  np.hstack((y_train_4, y_train_3))
    sm = SMOTE(random_state=27)
    X_train_6, y_train_6 = sm.fit_sample(X_train_5, y_train_5.ravel())
    X_train_6 = X_train_6[y_train_6==0]
    y_train_6 = y_train_6[y_train_6==0]
    return X_train_6, y_train_6

In [9]:
kfold = StratifiedKFold(n_splits=7, shuffle=True)
cvscores = []
y_train_2 = np.array(y_train_2)
X_train_2 = np.array(X_train_2)

for num_layers in [1,2,4]:
    for first_layer in [75, 25]:
        for loops in [3, 20]:
            for each_layer in [5, 10]:
                num_layers = num_layers
                first_layer = first_layer
                each_layer = each_layer
                num_smote = 1
                loops = loops
                scores = []
                for train, test in kfold.split(X_train_2, y_train_2):
                    print("num_layers", num_layers, "\nfirst_layer", first_layer, 
                          "\neach_layer", each_layer, "\nnum_smote", num_smote, "\nloops", loops)
                    x, y = create_data(X_train_2[train], y_train_2[train])
                    created_X = x
                    created_y = y
                    for i in range(loops):
                        X_train_3, y_train_3 = shuffle(X_train_2[train], y_train_2[train])
                        x, y = create_data(X_train_3, y_train_3)
                        created_X = np.vstack((created_X, x))
                        created_y = np.hstack((created_y, y))
                    X_train_3 = np.vstack([created_X, X_train_3])
                    y_train_3 =  np.hstack([created_y, y_train_3])

                    sm = SMOTE(random_state=27)
                    X_train_res, y_train_res = sm.fit_sample(X_train_3, y_train_3.ravel())

                  # create model
                    n_cols = X_train_res.shape[1]
                    model = Sequential()
                    model.add(Dense(first_layer, activation='elu', input_dim = n_cols))
                    model.add(Dropout(0.5))

                    for i in range(num_layers):
                        model.add(Dense(each_layer, activation='elu'))
                        model.add(Dropout(0.5))

                    model.add(Dense(1, activation='sigmoid'))

                    model.compile(optimizer='adam', 
                                  loss='binary_crossentropy')


                    # Fit the model
                    model.fit(X_train_res, y_train_res, epochs = 250, validation_split = .1, verbose=0)
                    # evaluate the model
                    y_pred =  model.predict(X_train_2[test])
                    y_pred = y_pred>0.5

                    s = f1_score(y_pred,y_train_2[test])
                    #print(s)
                    scores.append(s)
                print("mean scores", np.mean(scores))

num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 3
mean scores 0.145124716553288
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
num_layers 1 
first_layer 75 
each_layer 10 
num_smote 1 
loops 3
mean scores 0.13107606679035252
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20
num_layers 1 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20


...

C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


num_layers 2 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20
num_layers 2 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


num_layers 2 
first_layer 75 
each_layer 5 
num_smote 1 
loops 20


KeyboardInterrupt: 

In [ ]:
y_train_2[1]